In [11]:

import os
import sys
import glob
import argparse

import keras
import numpy as np

from keras.models  import Model
from keras.layers import  Input,Dense
from keras.callbacks import Callback
from keras.applications.inception_v3  import InceptionV3
train_dir ='../kagfish/train'


In [12]:
from keras.preprocessing.image import ImageDataGenerator


In [13]:
import wandb
from wandb.keras import WandbCallback
wandb.init()
config = wandb.config
config.img_width = 299
config.img_height = 299
config.epochs = 10
config.batch_size = 30

In [14]:
data_generator = ImageDataGenerator(rescale = 1./255,horizontal_flip=True,validation_split=0.2)

train_generator =data_generator.flow_from_directory("../kagfish/train",target_size=(config.img_width,config.img_height),
                                           batch_size=config.batch_size ,class_mode='categorical',subset="training")
val_generator =data_generator.flow_from_directory("../kagfish/train",target_size=(config.img_width,config.img_height),
                                           batch_size=config.batch_size ,class_mode='categorical',subset="validation")


Found 3025 images belonging to 8 classes.
Found 752 images belonging to 8 classes.


In [15]:
nb_train_samples=3025
nb_val_samples = 752

In [16]:
base =InceptionV3(weights ='imagenet',include_top=False)

In [17]:
from keras.models import Sequential
model = Sequential()
model.add(base)
model.add(keras.layers.GlobalAveragePooling2D(data_format=None))

In [18]:
model.add(Dense(8, activation='softmax'))

model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
inception_v3 (Model)         (None, None, None, 2048)  21802784  
_________________________________________________________________
global_average_pooling2d_2 ( (None, 2048)              0         
_________________________________________________________________
dense_2 (Dense)              (None, 8)                 16392     
Total params: 21,819,176
Trainable params: 21,784,744
Non-trainable params: 34,432
_________________________________________________________________


In [19]:
model.compile(loss='categorical_crossentropy',
              optimizer="adam",
              metrics=['accuracy'])

In [20]:
model.fit_generator(
    train_generator,
    epochs=config.epochs,
    steps_per_epoch = nb_train_samples // config.batch_size,
    validation_data=val_generator,
    validation_steps = nb_val_samples // config.batch_size,
    callbacks=[WandbCallback()])

Instructions for updating:
Use tf.cast instead.
Epoch 1/10


KeyboardInterrupt: 

In [ ]:
anno_classes = ['ALB', 'BET', 'DOL', 'LAG', 'NoF', 'OTHER', 'SHARK', 'YFT']
bb_json = {}
path = "bbox"
for c in anno_classes:
    if c == 'other': continue # no annotation file for "other" class
    j = json.load(open('{}/{}_labels.json'.format(path, c), 'r'))
    for l in j:
        if 'annotations' in l.keys() and len(l['annotations'])>0:
            bb_json[l['filename'].split('/')[-1]] = sorted(
                l['annotations'], key=lambda x: x['height']*x['width'])[-1]
bb_json['img_04908.jpg']

In [5]:
import glob
import numpy as np
folders = glob.glob('train/*')
from sklearn.model_selection import train_test_split
records = []
for folder in folders:
    files = glob.glob(folder+"/*.jpg")
    labels = [e.split('/')[1] for e in files]
    train, valid = train_test_split(range(len(files)), test_size=0.2, random_state=1983)
    mask = np.zeros(len(files))
    for j in train:
        mask[j] = 1
    for i, label in enumerate(labels):
        d = dict(label=label, file=files[i], train=mask[i])
        records.append(d)
import pandas as pd

from keras_preprocessing.image import ImageDataGenerator


import json
bb_json = {}
anno_classes = ['ALB', 'BET', 'DOL', 'LAG', 'OTHER', 'SHARK', 'YFT', 'NOF']
for c in anno_classes:
    j = json.load(open('bbox/{}_labels.json'.format(c), 'r'))
    for l in j:
        if 'annotations' in l.keys() and len(l['annotations'])>0:
            bb_json[l['filename'].split('/')[-1]] = sorted(
                l['annotations'], key=lambda x: x['height']*x['width'])[-1]
#print(bb_json.keys())
count=0
tot=0
keys = bb_json.keys()
records2 = []
for r in records:
    tot +=1
    name = r['file'].split('/')[-1]
    if not name in keys:
        count += 1
        #print(r['file'])
        r['bbox'] = None
    else:
        bbox = bb_json[name]
        r['x'] = bbox['x']
        r['y'] = bbox['y']
        r['width'] = bbox['width']
        r['height'] = bbox['height']
        records2.append(r)
print("nobbox", count, tot)
print("rec5", records2[:5])
df = pd.DataFrame.from_records(records2)
dftrain = df[df.train==1][['file', 'label', 'x', 'y', 'width', 'height']]
dfvalid = df[df.train==0][['file', 'label', 'x', 'y', 'width', 'height']]

dftrain.to_csv("tv_train.csv", index=False, header=True)
dfvalid.to_csv("tv_valid.csv", index=False, header=True)

FileNotFoundError: [Errno 2] No such file or directory: 'bbox/ALB_labels.json'